# Alternative Technical Indicators

---

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

## Import the CSV file and create the Pandas DataFrame

In [2]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/3035440181.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  blk_df = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


In [3]:
# Plot the DataFrame with hvplot
blk_df["close"].hvplot()

:Curve   [date]   (close)

---

## Compare the SMA50 plot line to an EMA50 plot line

In [4]:
# Create a DataFrame with the index and Close column from the dataset
signals_df = blk_df.loc[:, ["close"]].copy()

# Set the short window and long windows
short_window = 50

# Create a short window SMA
signals_df["SMA50"] = signals_df["close"].rolling(window=short_window).mean()

# Create a short window EMA
signals_df["EMA50"] = signals_df["close"].ewm(span=short_window).mean()

# Review the DataFrame
signals_df.iloc[45:55, :]

,close,SMA50,EMA50
date,,,
2018-03-08,553.39,NaN,547.078283
2018-03-09,577.18,NaN,548.471241
2018-03-12,571.79,NaN,549.542753
2018-03-13,564.06,NaN,550.205367
2018-03-14,558.96,549.7112,550.602405
2018-03-15,557.93,550.6904,550.932697
2018-03-16,563.66,551.6768,551.503041
2018-03-19,553.83,552.3308,551.606737
2018-03-20,557.52,552.9698,551.868849


In [5]:
# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["SMA50", "EMA50"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
sma_ema_plot = security_close * moving_avgs
sma_ema_plot

:Overlay
   .Curve.I     :Curve   [date]   (close)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [date]   (value)

---

## Create a trading algorithm using EMA technical indicators

In [6]:
# Create a DataFrame with the index and Close column from the dataset
ema_signals_df = blk_df.loc[:, ["close"]].copy()

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
ema_signals_df["EMA50"] = signals_df["close"].ewm(span=short_window).mean()
ema_signals_df["EMA100"] = signals_df["close"].ewm(span=long_window).mean()

# Review the DataFrame
ema_signals_df.head()

,close,EMA50,EMA100
date,,,
2018-01-02,508.97,508.970000,508.970000
2018-01-03,514.34,511.708700,511.681850
2018-01-04,521.13,514.975571,514.894429
2018-01-05,525.57,517.785171,517.643917
2018-01-08,529.49,520.317082,520.108840


In [7]:
# Set the Signal column
ema_signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the EMA50 is greater than (or crosses over) the EMA100
# where 0 is when the EMA50 is under the EMA100
ema_signals_df["Signal"][short_window:] = np.where(
    ema_signals_df["EMA50"][short_window:] > ema_signals_df["EMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
ema_signals_df["Entry/Exit"] = ema_signals_df["Signal"].diff()

# Review the DataFrame
ema_signals_df.head()

,close,EMA50,EMA100,Signal,Entry/Exit
date,,,,,
2018-01-02,508.97,508.970000,508.970000,0.0,NaN
2018-01-03,514.34,511.708700,511.681850,0.0,0.0
2018-01-04,521.13,514.975571,514.894429,0.0,0.0
2018-01-05,525.57,517.785171,517.643917,0.0,0.0
2018-01-08,529.49,520.317082,520.108840,0.0,0.0


In [8]:
# Visualize entry position relative to close price
entry = ema_signals_df[ema_signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = ema_signals_df[ema_signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = ema_signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = ema_signals_df[["EMA50", "EMA100"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

---

## Incorporate the finta Python library into the DMAC trading algorithm

In [10]:
%pip install finta

  Using cached finta-1.3-py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import the finta library
from finta import TA

In [13]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/3035440181.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  blk_df = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


### Generate the EMA Technical Indicator using FinTA

In [17]:
# Set the short window and long windows
short_window = 50
long_window = 100

# Add the EMA technical indicators for the short and long windows
blk_df["Short"] = TA.EMA(blk_df, short_window)
blk_df["Long"] = TA.EMA(blk_df, long_window)

# Review the DataFrame
blk_df.iloc[45:105, :]

,open,high,low,close,volume,Short,Long
date,,,,,,,
2018-03-08,548.87,554.37,547.02,553.39,595221,547.078283,547.689520
2018-03-09,558.33,578.65,558.33,577.18,814709,548.471241,548.647815
2018-03-12,578.34,578.80,569.01,571.79,410921,549.542753,549.390396
2018-03-13,577.26,578.56,562.18,564.06,477530,550.205367,549.855398
2018-03-14,566.66,566.66,557.56,558.96,341448,550.602405,550.140606
2018-03-15,560.25,563.63,555.85,557.93,313222,550.932697,550.381834
2018-03-16,558.79,565.88,558.04,563.66,616887,551.503041,550.788501
2018-03-19,563.85,563.98,548.09,553.83,554578,551.606737,550.880655
2018-03-20,557.54,557.84,552.00,557.52,381157,551.868849,551.079730


In [18]:
# Set the Signal column
blk_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
blk_df["Signal"][short_window:] = np.where(
    blk_df["Short"][short_window:] > blk_df["Long"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
blk_df["Entry/Exit"] = blk_df["Signal"].diff()

# Review the DataFrame
blk_df.iloc[95:105, :]

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/2501863730.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blk_df["Signal"][short_window:] = np.where(


,open,high,low,close,volume,Short,Long,Signal,Entry/Exit
date,,,,,,,,,
2018-05-18,536.28,539.92,534.42,538.49,518719,532.020294,535.658182,0.0,0.0
2018-05-21,542.00,547.98,542.00,543.48,359081,532.479165,535.839061,0.0,0.0
2018-05-22,546.44,549.58,545.86,546.88,406234,533.055330,536.093536,0.0,0.0
2018-05-23,541.72,542.79,533.75,542.66,377715,533.439299,536.244392,0.0,0.0
2018-05-24,542.94,542.94,534.68,540.83,413054,533.734535,536.349408,0.0,0.0
2018-05-25,537.14,540.32,535.94,539.61,290043,533.969071,536.423848,0.0,0.0
2018-05-29,534.75,537.82,518.03,524.26,714247,533.581778,536.146982,0.0,0.0
2018-05-30,527.31,535.29,526.07,534.29,404142,533.610010,536.104839,0.0,0.0
2018-05-31,527.31,535.29,526.07,534.29,83,533.637099,536.063771,0.0,0.0


In [19]:
# Visualize entry position relative to close price
entry = blk_df[blk_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = blk_df[blk_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = blk_df[["Short", "Long"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

## Demonstrate Alternative Technical Indicators Using FinTA

### Smoothed Moving Average (SMMA) Demo

#### Loading the OHLCV Data to Create a Fresh DataFrame

In [20]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/3035440181.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  blk_df = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


#### Computing SMMA

In [21]:
# Add the SMMA technical indicators for the short and long windows
blk_df["Short"] = TA.SMMA(blk_df, short_window)
blk_df["Long"] = TA.SMMA(blk_df, long_window)

# Review the DataFrame
blk_df.head()

,open,high,low,close,volume,Short,Long
date,,,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135,508.970000,508.970000
2018-01-03,508.66,514.74,508.16,514.34,448724,511.682121,511.668492
2018-01-04,516.50,521.31,515.01,521.13,389830,514.895248,514.854078
2018-01-05,523.86,525.94,520.89,525.57,482968,517.645345,517.573580
2018-01-08,523.94,529.72,523.52,529.49,561517,520.110947,520.005009


#### Running the Trading Algorithm Using SMMA as Technical Indicator

In [22]:
# Set the Signal column
blk_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
blk_df["Signal"][short_window:] = np.where(
    blk_df["Short"][short_window:] > blk_df["Long"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
blk_df["Entry/Exit"] = blk_df["Signal"].diff()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/1646486707.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blk_df["Signal"][short_window:] = np.where(


#### Visualizing the SMMA Entry/Exit Signals

In [24]:
# Visualize entry position relative to close price
entry = blk_df[blk_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = blk_df[blk_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = blk_df[["Short", "Long"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot1 = security_close * moving_avgs * entry * exit
entry_exit_plot1

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

### Volume-Adjusted Moving Average (SMMA) Demo

#### Loading the OHLCV Data to Create a Fresh DataFrame

In [27]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/3035440181.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  blk_df = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


#### Computing VAMA

In [28]:
# Add the VAMA technical indicators for the short and long windows
blk_df["Short"] = TA.VAMA(blk_df, short_window)
blk_df["Long"] = TA.VAMA(blk_df, long_window)

# Review the DataFrame
blk_df.head()

,open,high,low,close,volume,Short,Long
date,,,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135,NaN,NaN
2018-01-03,508.66,514.74,508.16,514.34,448724,NaN,NaN
2018-01-04,516.50,521.31,515.01,521.13,389830,NaN,NaN
2018-01-05,523.86,525.94,520.89,525.57,482968,NaN,NaN
2018-01-08,523.94,529.72,523.52,529.49,561517,NaN,NaN


#### Running the Trading Algorithm Using VAMA as Technical Indicator

In [29]:
# Set the Signal column
blk_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
blk_df["Signal"][short_window:] = np.where(
    blk_df["Short"][short_window:] > blk_df["Long"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
blk_df["Entry/Exit"] = blk_df["Signal"].diff()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/1646486707.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blk_df["Signal"][short_window:] = np.where(


#### Visualizing the VAMA Entry/Exit Signals

In [30]:
# Visualize entry position relative to close price
entry = blk_df[blk_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = blk_df[blk_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = blk_df[["Short", "Long"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

## Demonstrate Alternative Trading Signals - Bollinger Bands

### Loading the OHLCV Data to Create a Fresh DataFrame¶

In [31]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
blk_df = pd.read_csv(
    Path("../Resources/blk_ohlcv.csv"),
    index_col = "date", 
    parse_dates = True, 
    infer_datetime_format = True
)

# Review the DataFrame
blk_df.head()

/var/folders/h0/ckkxq40s70vc524w2v0_myw00000gp/T/ipykernel_74777/3035440181.py:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  blk_df = pd.read_csv(


,open,high,low,close,volume
date,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135
2018-01-03,508.66,514.74,508.16,514.34,448724
2018-01-04,516.50,521.31,515.01,521.13,389830
2018-01-05,523.86,525.94,520.89,525.57,482968
2018-01-08,523.94,529.72,523.52,529.49,561517


### Computing the Bollinger Bands

In [32]:
# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(blk_df)

# Review the DataFrame
bbands_df.iloc[17:25, :]

,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,
2018-01-26,NaN,NaN,NaN
2018-01-29,NaN,NaN,NaN
2018-01-30,614.168935,556.8535,499.538065
2018-01-31,612.202971,559.4950,506.787029
2018-02-01,610.291851,562.0375,513.783149
2018-02-02,608.207884,563.3970,518.586116
2018-02-05,608.285188,563.3670,518.448812
2018-02-06,607.727046,563.6535,519.579954


### Visualizing the Bollinger Bands

In [33]:
# Concatenate the Bollinger Bands to the DataFrame
blk_df = pd.concat([blk_df, bbands_df], axis=1)

# Review the DataFrame
blk_df.iloc[17:25, :]

,open,high,low,close,volume,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,
2018-01-26,578.21,586.95,577.12,586.80,516601,NaN,NaN,NaN
2018-01-29,585.53,585.53,574.20,574.42,611014,NaN,NaN,NaN
2018-01-30,570.05,571.53,562.18,562.29,851135,614.168935,556.8535,499.538065
2018-01-31,564.81,566.97,559.65,561.80,665825,612.202971,559.4950,506.787029
2018-02-01,561.41,568.48,558.75,565.19,659597,610.291851,562.0375,513.783149
2018-02-02,565.26,565.26,545.81,548.32,808150,608.207884,563.3970,518.586116
2018-02-05,539.99,553.03,518.50,524.97,1129441,608.285188,563.3670,518.448812
2018-02-06,510.81,538.52,507.26,535.22,1076684,607.727046,563.6535,519.579954


In [34]:
# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_upper = blk_df[["BB_UPPER"]].hvplot(
    line_color="purple",
    ylabel="Price in $",
    width=1000,
    height=400
)


bb_middle = blk_df[["BB_MIDDLE"]].hvplot(
    line_color="orange",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_lower = blk_df[["BB_LOWER"]].hvplot(
    line_color="blue",
    ylabel="Price in $",
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower
bbands_plot

:Overlay
   .Curve.I   :Curve   [date]   (close)
   .Curve.II  :Curve   [date]   (BB_UPPER)
   .Curve.III :Curve   [date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [date]   (BB_LOWER)

### Creating a Trading Algorithm Using Bollinger Bands

In [35]:
# Create a trading algorithm using Bollinger Bands
# Set the Signal column
blk_df["Signal"] = 0.0

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
for index, row in blk_df.iterrows():
    if row["close"] < row["BB_LOWER"]:
        blk_df.loc[index, "Signal"] = 1.0
    if row["close"] > row["BB_UPPER"]:
        blk_df.loc[index,"Signal"] = -1.0

# Review the DataFrame
blk_df

,open,high,low,close,volume,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
date,,,,,,,,,
2018-01-02,518.78,518.78,506.48,508.97,527135,NaN,NaN,NaN,0.0
2018-01-03,508.66,514.74,508.16,514.34,448724,NaN,NaN,NaN,0.0
2018-01-04,516.50,521.31,515.01,521.13,389830,NaN,NaN,NaN,0.0
2018-01-05,523.86,525.94,520.89,525.57,482968,NaN,NaN,NaN,0.0
2018-01-08,523.94,529.72,523.52,529.49,561517,NaN,NaN,NaN,0.0
2018-01-09,529.00,540.00,528.57,533.94,547671,NaN,NaN,NaN,0.0
2018-01-10,534.47,535.88,529.78,531.57,483772,NaN,NaN,NaN,0.0
2018-01-11,534.15,538.96,530.61,537.92,765925,NaN,NaN,NaN,0.0
2018-01-12,552.70,556.37,546.71,555.53,941849,NaN,NaN,NaN,0.0


### Visualizing the Entry/Exit Positions using Bollinger Bands

In [36]:
# Visualize entry position relative to close price
entry = blk_df[blk_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = blk_df[blk_df["Signal"] == -1.0]["close"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = blk_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_upper = blk_df[["BB_UPPER"]].hvplot(
    line_color="purple",
    ylabel="Price in $",
    width=1000,
    height=400
)


bb_middle = blk_df[["BB_MIDDLE"]].hvplot(
    line_color="orange",
    ylabel="Price in $",
    width=1000,
    height=400
)

bb_lower = blk_df[["BB_LOWER"]].hvplot(
    line_color="blue",
    ylabel="Price in $",
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .Curve.II         :Curve   [date]   (BB_UPPER)
   .Curve.III        :Curve   [date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [date]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

In [33]:
# Set the Signal column
ema_signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the EMA50 is greater than (or crosses over) the EMA100
# where 0 is when the EMA50 is under the EMA100
ema_signals_df["Signal"][short_window:] = np.where(
    ema_signals_df["EMA50"][short_window:] > ema_signals_df["EMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
ema_signals_df["Entry/Exit"] = ema_signals_df["Signal"].diff()

# Review the DataFrame
ema_signals_df.head()


,close,EMA50,EMA100,Signal,Entry/Exit
date,,,,,
2018-01-02,508.97,508.970000,508.970000,0.0,NaN
2018-01-03,514.34,511.708700,511.681850,0.0,0.0
2018-01-04,521.13,514.975571,514.894429,0.0,0.0
2018-01-05,525.57,517.785171,517.643917,0.0,0.0
2018-01-08,529.49,520.317082,520.108840,0.0,0.0
